In [1]:
import numpy as np
import pandas as pd
import string
#import xgboost as xgb
import io
import nltk
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')

from textblob import TextBlob
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Datasets

In [2]:
tweets = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
tweets['keyword'] = tweets.keyword.str.replace('%20',' ')
test['keyword'] = test.keyword.str.replace('%20',' ')

In [3]:
tweets.drop_duplicates(subset = 'text', keep = False, inplace = True)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7434 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7434 non-null   int64 
 1   keyword   7378 non-null   object
 2   location  4982 non-null   object
 3   text      7434 non-null   object
 4   target    7434 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 348.5+ KB


In [4]:
test['text'].duplicated().value_counts()

False    3243
True       20
Name: text, dtype: int64

### Extra Features

In [5]:
sia = SentimentIntensityAnalyzer()
def return_sia_compound_values(text):
    return sia.polarity_scores(text)['compound']

In [6]:
def remove_stopword(text):
    new_text = []
    for e in text:
        if e not in stopwords and e.isalpha():
            new_text.append(e)
    text = new_text
    return " ".join(new_text)

def stemm(text):
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

def contains_punctuation(text):
    punctuation = set(string.punctuation)
    for character in text:
        if character in punctuation:
            return True
    return False

def amount_of_punctuation(text):
    punctuation = set(string.punctuation)
    amount = 0
    for character in text:
        if character in punctuation: amount += 1
    return amount

def get_adjectives(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("JJ")])

def get_nouns(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("NN")])

def get_verbs(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("VB")])

def get_adverbs(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("RB")])

In [7]:
def get_metrics(df):
    tweets_metrics = df[['id','text']]
    tweets_metrics['text_without_stopwords'] = tweets_metrics['text'].str.split()
    tweets_metrics['text_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(remove_stopword)
    tweets_metrics['length'] = tweets_metrics['text'].apply(lambda x: len(x))
    tweets_metrics['avg_word_length'] = tweets_metrics['text'].str.split().apply(lambda x: [len(y) for y in x]).transform(lambda x: np.mean(x))
    tweets_metrics['amount_of_words'] = tweets_metrics['text'].str.split().transform(lambda x: len(x))
    unique_words_by_tweet = tweets_metrics['text'].transform(lambda x: x.split()).transform(lambda x: pd.Series(x).unique()).transform(lambda x: len(x))
    tweets_metrics['amount_of_unique_words'] = unique_words_by_tweet
    tweets_metrics['sentiment'] = tweets_metrics['text'].apply(lambda x: return_sia_compound_values(x))
    tweets_metrics['stopwords_count'] = tweets_metrics['text'].apply(lambda x: len([word for word in str(x).lower().split() if word in stopwords]))
    tweets_metrics['punctuation_count'] = tweets_metrics['text'].apply(lambda x: amount_of_punctuation(x))
    mentions = tweets_metrics['text'].str.findall(r'@.\S*?(?=\s|[:]|$)').to_frame()
    tweets_metrics['mentions_count'] = mentions['text'].apply(lambda x: len(x))
    hashtags = tweets_metrics['text'].str.findall(r'#[^?\s].*?(?=\s|$)')
    tweets_metrics['hashtags_count'] = hashtags.apply(lambda x: len(x))
    tweets_metrics['longest_word_length_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(lambda x: ([len(word) for word in str(x).lower().split() if not word.startswith('http')])).apply(lambda x: max(x) if len(x) > 0 else 0)
    tweets_metrics['stopword_word_ratio'] = tweets_metrics['stopwords_count'] / tweets_metrics['amount_of_words']
    tweets_metrics['adjectives_count'] = tweets_metrics['text'].apply(get_adjectives)
    tweets_metrics['nouns_count'] = tweets_metrics['text'].apply(get_nouns)
    tweets_metrics['verbs_count'] = tweets_metrics['text'].apply(get_verbs)
    tweets_metrics['adverbs_count'] = tweets_metrics['text'].apply(get_adverbs)
    return tweets_metrics

In [8]:
train_metrics = get_metrics(tweets)

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:

In [9]:
test_metrics = get_metrics(test)

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
# Run for files
train_metrics.to_csv('train_features.csv', index=False)
test_metrics.to_csv('test_features.csv', index=False)

### Text preprocessing

In [11]:
# Mocks real tokenizer used in glove
import re

def tokenize_input(input_text):
    to_tokens = input_text[:]
    token_specification = [
        ('url', r'https?:\/\/\S+\b|www\.(\w+\.)+\S*'),
        (' / ', r'/'),
        ('user', r'@\w+'),            
        ('smile', r'[8:=;][)d]+|[)d]+[\'`\-]?[8:=;]'),    
        ('lolface', r'[8:=;][\'`\-]?p'),      
        ('sadface', r'[8:=;][\'`\-]?\(|\)+[8:=;][\'`\-]?'),          
        ('neutralface', r'[8:=;][\'`\-]?[\/|l*]'),       
        ('heart', r'<3'),   
        ('number', r'[-+]?[.\d]*[\d]+[:,.\d]*')
    ]
    for replacement, regex in token_specification:
        to_tokens = re.sub(regex, replacement, to_tokens)
    return to_tokens

In [12]:
# Check
tokenize_input('https://regexr.com hola / :) <3 :p :(  8888 @justin')

'url hola  /  smile heart lolface sadface  number user'

In [13]:
import string

tweets['text'] = tweets['text'].apply(tokenize_input)
test['text'] = test['text'].apply(tokenize_input)
tweets['text'] = tweets['text'].apply(lambda x: x.translate({ord(i): ' ' for i in string.punctuation}))
test['text'] = test['text'].apply(lambda x: x.translate({ord(i): ' ' for i in string.punctuation}))
tweets['text'] = tweets['text'].apply(lambda x: x.lower())
test['text'] = test['text'].apply(lambda x: x.lower())

In [14]:
# Run for files
tweets.to_csv('processed_train.csv', index=False)
test.to_csv('processed_test.csv', index=False)

### Vector encoding

#### Keywords

In [15]:
tweets['keyword'] = tweets['keyword'].fillna('null')
test['keyword'] = test['keyword'].fillna('null')

In [16]:
keyword_tokens = tweets.keyword.unique().tolist()
keyword_test = test.keyword.unique().tolist()

In [17]:
for k in keyword_test:
    if k not in keyword_tokens:
        print(k) # Mismas palabras en ambos sets

In [18]:
embeddings_index = dict()
f = open('glove.6B.100d.txt') # Vectores entrenados de 100 dimensiones
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400001 word vectors.


In [19]:
vectors = {}
for keyword in keyword_tokens:
    words = keyword.split(' ')
    n = len(words)
    if n == 1:
        vectors[keyword] = embeddings_index[keyword]
    else:
        acum = np.zeros(100)
        for w in words:
            acum = np.sum([acum,embeddings_index[w]] , axis=0)
        vectors[keyword] = acum

In [20]:
keyword_vectors = pd.DataFrame.from_dict(vectors).T.reset_index()
keyword_vectors.head()

,index,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,null,0.079432,-0.14054,-0.104620,-0.362590,-0.22721,-0.13612,0.74755,0.328090,0.54364,...,-1.21230,0.515730,0.16573,0.679430,0.353270,0.17672,0.258030,0.068445,-1.20160,-0.20168
1,ablaze,0.137010,-0.31349,-0.047427,-0.245820,0.76459,1.21330,0.25674,0.446160,1.21040,...,1.42780,-0.402050,-0.26682,-0.029039,-1.102300,0.20442,-0.064528,0.305040,0.42830,0.54531
2,accident,-0.006329,-0.37913,0.409920,-0.003844,-0.81139,-0.67840,0.25995,1.090300,0.60039,...,0.49451,-0.308700,-0.18550,0.714090,0.198860,1.12760,-0.100960,-0.100000,0.21349,-1.24530
3,aftershock,0.136920,1.02570,0.539610,0.275310,-0.91579,0.24287,0.77162,0.025242,0.47416,...,1.23600,0.126510,-0.93994,0.187410,0.712540,0.79876,-0.040149,-0.591220,-0.28051,-0.23293
4,airplane accident,-0.175379,-0.10526,0.977860,-0.001856,-0.93397,-1.15404,0.79570,1.143708,0.94012,...,0.78136,-0.287935,0.23436,0.676289,0.262775,1.53821,-0.300710,-0.089367,0.77883,-2.21178


In [21]:
aux = []
aux.append('keyword')
for i in range (0, 100):
    name = 'k' + str(i)
    aux.append(name)

In [23]:
keyword_vectors.columns = aux
keyword_vectors.head()

,keyword,k0,k1,k2,k3,k4,k5,k6,k7,k8,...,k90,k91,k92,k93,k94,k95,k96,k97,k98,k99
0,null,0.079432,-0.14054,-0.104620,-0.362590,-0.22721,-0.13612,0.74755,0.328090,0.54364,...,-1.21230,0.515730,0.16573,0.679430,0.353270,0.17672,0.258030,0.068445,-1.20160,-0.20168
1,ablaze,0.137010,-0.31349,-0.047427,-0.245820,0.76459,1.21330,0.25674,0.446160,1.21040,...,1.42780,-0.402050,-0.26682,-0.029039,-1.102300,0.20442,-0.064528,0.305040,0.42830,0.54531
2,accident,-0.006329,-0.37913,0.409920,-0.003844,-0.81139,-0.67840,0.25995,1.090300,0.60039,...,0.49451,-0.308700,-0.18550,0.714090,0.198860,1.12760,-0.100960,-0.100000,0.21349,-1.24530
3,aftershock,0.136920,1.02570,0.539610,0.275310,-0.91579,0.24287,0.77162,0.025242,0.47416,...,1.23600,0.126510,-0.93994,0.187410,0.712540,0.79876,-0.040149,-0.591220,-0.28051,-0.23293
4,airplane accident,-0.175379,-0.10526,0.977860,-0.001856,-0.93397,-1.15404,0.79570,1.143708,0.94012,...,0.78136,-0.287935,0.23436,0.676289,0.262775,1.53821,-0.300710,-0.089367,0.77883,-2.21178


In [24]:
keywords_to_merge = tweets.merge(keyword_vectors, how='left').drop(columns=['keyword', 'location', 'text', 'target'])

In [25]:
# Run for files
#keywords_to_merge.to_csv('keyword_features.csv', index=False)

### Keywords one hot encoding

In [27]:
tweets

,id,keyword,location,text,target
0,1,null,NaN,our deeds are the reason of this earthquake m...,1
1,4,null,NaN,forest fire near la ronge sask canada,1
2,5,null,NaN,all residents asked to shelter in place are ...,1
3,6,null,NaN,number people receive wildfires evacuation or...,1
4,7,null,NaN,just got sent this photo from ruby alaska as ...,1
...,...,...,...,...,...
7604,10863,null,NaN,worldnews fallen powerlines on gneutralfacein...,1
7605,10864,null,NaN,on the flip side i m at walmart and there is a...,1
7606,10866,null,NaN,suicide bomber kills number in saudi security ...,1
7608,10869,null,NaN,two giant cranes holding a bridge collapse int...,1


In [28]:
disaster_list = list(tweets['keyword'].unique())

In [29]:
crash = ['collide', 'collided', 'collision', 'crash', 'crashed', 'wreck', 'wreckage', 'wrecked']

emergency = ['emergency', 'emergency plan']

electricity = ['electrocute', 'electrocuted',]

helpers = ['ambulance', 'police', 'siren', 'sirens', 'emergency services', 'first responders',\
           'stretcher', 'eyewitness', 'rescuers']

panic = ['screamed', 'screaming', 'screams', 'panic', 'mayhem', 'riot', 'rioting', 'fear', 'panicking', 'trauma',\
         'trouble', 'hail', 'pandemonium']

hostages = ['hostage', 'hostages', 'trapped']

quarentine = ['quarantine', 'quarantined']

colapse = ['bridge collapse', 'collapse', 'collapsed', 'demolish', 'demolished', 'demolition', 'structural failure']

accident = ['accident', 'airplane accident', 'derail', 'derailed', 'derailment', 'oil spill']

fire = ['ablaze', 'hellfire', 'smoke', 'wild fires', 'wildfire', 'buildings burning',\
        'buildings on fire', 'burned', 'burning', 'burning buildings', 'bush fires', 'fire',\
        'fire truck', 'flames', 'forest fire', 'forest fires', 'blaze', 'blazing', 'arson', 'arsonist']

nuclear = ['nuclear disaster', 'nuclear reactor', 'radiation emergency', 'meltdown']

explotion = ['explode', 'exploded', 'explosion', 'blown up', 'blew up', 'loud bang']

survivor = ['survive', 'survived', 'rescue', 'rescued', 'survivors', 'evacuate', 'evacuated', 'evacuation', 'refugees']

wounded = ['wounded', 'wounds', 'bleeding', 'bloody', 'injured', 'injuries', 'injury', 'traumatised', 'blood']

bomb = ['suicide bomb', 'suicide bomber', 'suicide bombing', 'bomb', 'bombed', 'bombing', 'detonate', 'detonation']

storm = ['storm', 'thunderstorm', 'thunder', 'rainstorm', 'violent storm', 'windstorm', 'lightning', 'hailstorm']

water = ['flood', 'flooding', 'floods', 'inundated', 'inundation', 'sinking', 'drown', 'drowned', 'drowning', 'sunk']

natural_disaster = ['heat wave','sandstorm', 'seismic' ,'avalanche', 'tsunami', 'twister',\
                    'typhoon',  'tornado', 'hurricane', 'natural disaster', 'cyclone', 'volcano',\
                    'drought', 'dust storm', 'earthquake',  'lava', 'aftershock', 'snowstorm', 'blizzard',\
                    'whirlwind', 'upheaval',  'landslide', 'cliff fall', 'mudslide', 'sinkhole', 'displaced',\
                    'epicentre']

attack = ['attack', 'attacked']

casualties = ['mass murder', 'mass murderer', 'massacre', 'fatal', 'fatalities', 'fatality', 'casualties',\
              'casualty', 'body bag', 'body bagging', 'body bags', 'dead', 'death', 'deaths',  'tragedy']

terrorism = ['terrorism', 'terrorist', 'threat', 'hijack', 'hijacker', 'hijacking', 'bioterror', 'bioterrorism']

destruction = ['destroyed', 'destruction', 'devastated',\
               'devastation', 'disaster', 'annihilated', 'annihilation', 'apocalypse',\
               'armageddon', 'catastrophe', 'catastrophic', 'obliterate', 'obliterated',\
               'obliteration', 'damage', 'destroy', 'desolate', 'desolation', 'blight',\
               'harm', 'hazard', 'hazardous', 'danger', 'ruin', 'engulfed', 'rubble', 'debris',\
               'razed', 'flattened', 'crush', 'crushed']

warlike = ['war zone', 'weapon', 'weapons', 'military', 'army', 'battle', 'outbreak', 'chemical emergency', 'curfew']

starvation = ['famine', 'deluge', 'deluged']


null = ['null']

In [30]:
disasters = {'crash' : crash, 'emergency' : emergency, 'electricity': electricity,\
             'helpers': helpers, 'panic' : panic,  'hostages' : hostages, 'quarentine' : quarentine,\
             'colapse' : colapse, 'accident' : accident, 'fire' : fire,\
             'nuclear' : nuclear, 'explotion' : explotion, 'survivor' : survivor,\
             'wounded' : wounded, 'bomb' : bomb, 'storm' : storm,\
             'water' : water, 'natural_disaster' : natural_disaster, 'attack' : attack,\
             'casualties' : casualties, 'terrorism' : terrorism, 'destruction' : destruction,\
             'warlike' : warlike, 'starvation' : starvation, 'null' : null}

In [31]:
n_categories = len(disasters)
n_categories

25

In [32]:
numeric_maps = {}
count = 0
for k in disasters.keys():
    numeric_maps[k] = count
    count += 1

In [33]:
mapping = {}
for k, v in disasters.items():
    for w in v:
        mapping[w] = k

In [34]:
mapping_df = pd.DataFrame.from_dict(mapping, orient='index').reset_index()
mapping_df.columns = ['keyword', 'group']
numeric_maps_df = pd.DataFrame.from_dict(numeric_maps, orient='index').reset_index()
numeric_maps_df.columns = ['group', 'id']
to_encode = mapping_df.merge(numeric_maps_df)

In [35]:
to_encode

,keyword,group,id
0,collide,crash,0
1,collided,crash,0
2,collision,crash,0
3,crash,crash,0
4,crashed,crash,0
...,...,...,...
217,curfew,warlike,22
218,famine,starvation,23
219,deluge,starvation,23
220,deluged,starvation,23


In [36]:
from sklearn.preprocessing import LabelBinarizer

lb = preprocessing.LabelBinarizer()
lb.fit(range(0, n_categories))

LabelBinarizer()

In [37]:
lb.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [38]:
to_encode['encode'] = list(lb.transform(to_encode['id']))

In [39]:
expanded = to_encode.encode.apply(pd.Series)

In [40]:
result = pd.concat([to_encode, expanded], axis=1, sort=False)

In [41]:
result.drop(columns=['encode', 'id'], inplace=True)

In [42]:
result.head()

,keyword,group,0,1,2,3,4,5,6,7,...,15,16,17,18,19,20,21,22,23,24
0,collide,crash,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,collided,crash,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,collision,crash,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,crash,crash,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,crashed,crash,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
keyword_exp = tweets.loc[:, ['id', 'keyword']]
keyword_exp_test = test.loc[:, ['id', 'keyword']]

In [44]:
merged_train = keyword_exp.merge(result, left_on='keyword', right_on='keyword', how='left')
merged_test = keyword_exp_test.merge(result, left_on='keyword', right_on='keyword', how='left')

In [45]:
merged_train.drop(columns=['keyword', 'group'], inplace=True)
merged_test.drop(columns=['keyword', 'group'], inplace=True)

In [46]:
# Run for file
#merged_train.to_csv('keyword_mapping.csv', index=False)
#merged_test.to_csv('keyword_mapping_test.csv', index=False)

### Location

In [47]:
locations = pd.read_csv("../TP1/locations.csv", usecols=['location', 'address', 'point'])
locations.head()

,location,address,point
0,,NaN,NaN
1,glasgow,"Glasgow, Glasgow City, Scotland, G2 9SA, Unite...","(55.8609825, -4.2488787, 0.0)"
2,"melbourne, australia","City of Melbourne, Victoria, Australia","(-37.8142176, 144.9631608, 0.0)"
3,news,"34375, Abbotsford Centre, Abbotsford, Fraser V...","(49.04172215, -122.27255349013137, 0.0)"
4,alberta,"Alberta, Canada","(55.001251, -115.002136, 0.0)"


In [48]:
locations.fillna('null', inplace=True)
empty_loc = locations.loc[0, 'location']
locations.replace(empty_loc, 'null', inplace=True)
locations.head()

,location,address,point
0,null,null,null
1,glasgow,"Glasgow, Glasgow City, Scotland, G2 9SA, Unite...","(55.8609825, -4.2488787, 0.0)"
2,"melbourne, australia","City of Melbourne, Victoria, Australia","(-37.8142176, 144.9631608, 0.0)"
3,news,"34375, Abbotsford Centre, Abbotsford, Fraser V...","(49.04172215, -122.27255349013137, 0.0)"
4,alberta,"Alberta, Canada","(55.001251, -115.002136, 0.0)"


#### Coordinates

In [49]:
tweets['location'] = tweets['location'].fillna('null')
test['location'] = test['location'].fillna('null')

In [50]:
def point_to_list(point):
    if point == 'null':
        return [300.0, 300.0] # Arbitrary large number 
    
    coordinates = []
    aux = point[:]
    row = aux.strip( '()' ).split(',')
    coordinates.append(float(row[0]))
    coordinates.append(float(row[1]))
    return coordinates

In [51]:
locations['point'] = locations.point.apply(point_to_list)

In [52]:
aux = locations.point.apply(pd.Series)
aux.columns = ['x', 'y']

In [53]:
locations['x'] = aux['x']
locations['y'] = aux['y']

In [54]:
locations.head()

,location,address,point,x,y
0,null,null,"[300.0, 300.0]",300.000000,300.000000
1,glasgow,"Glasgow, Glasgow City, Scotland, G2 9SA, Unite...","[55.8609825, -4.2488787]",55.860982,-4.248879
2,"melbourne, australia","City of Melbourne, Victoria, Australia","[-37.8142176, 144.9631608]",-37.814218,144.963161
3,news,"34375, Abbotsford Centre, Abbotsford, Fraser V...","[49.04172215, -122.27255349013137]",49.041722,-122.272553
4,alberta,"Alberta, Canada","[55.001251, -115.002136]",55.001251,-115.002136


In [55]:
tweets['location'] = tweets['location'].apply(str.lower)
test['location'] = test['location'].apply(str.lower)

In [56]:
coordinates_train = tweets.merge(locations.loc[:, ['location', 'x', 'y']], left_on='location', right_on='location', how='left').loc[:, ['id', 'x', 'y']]
coordinates_test = test.merge(locations.loc[:, ['location', 'x', 'y']], left_on='location', right_on='location', how='left').loc[:, ['id', 'x', 'y']]

In [57]:
# Run for file
# metrics_with_xy.to_csv('coordinates_train.csv', index=False)
# test_metrics_with_xy.to_csv('coordinates_test.csv', index=False)